### Nonlocal Scopes

Функции, определенные внутри другой функции, могут ссылаться на переменные из этой охватывающей области действия, точно так же, как функции могут ссылаться на переменные из глобальной области действия.

In [1]:
def outer_func():
    x = 'hello'

    def inner_func():
        print(x)

    inner_func()

In [2]:
outer_func()

hello


Фактически, поддерживается любой уровень вложенности, поскольку Python просто продолжает поиск во вложенных областях видимости до тех пор, пока не найдет то, что ему нужно (или не найдет это к моменту завершения поиска во встроенной области видимости, в этом случае возникает ошибка времени выполнения).

In [3]:
def outer_func():
    x = 'hello'
    def inner1():
        def inner2():
            print(x)
        inner2()
    inner1()

In [4]:
outer_func()

hello


Но если мы **присваиваем** значение переменной, оно считается частью локальной области видимости и потенциально **маскирует** имена переменных области видимости enclsogin:

In [5]:
def outer():
    x = 'hello'
    def inner():
        x = 'python'
    inner()
    print(x)

In [6]:
outer()

hello


Как видите, **x** в **outer** не изменился.

Чтобы добиться этого, мы можем использовать ключевое слово **nonlocal**:

In [7]:
def outer():
    x = 'hello'
    def inner():
        nonlocal x
        x = 'python'
    inner()
    print(x)

In [8]:
outer()

python


Конечно, это может работать на любом уровне:

In [9]:
def outer():
    x = 'hello'

    def inner1():
        def inner2():
            nonlocal x
            x = 'python'
        inner2()
    inner1()
    print(x)

In [10]:
outer()

python


Насколько далеко Python просматривает цепочку, зависит от первого появления имени переменной в охватывающей области видимости.

Рассмотрим следующий пример:

In [11]:
def outer():
    x = 'hello'
    def inner1():
        x = 'python'
        def inner2():
            nonlocal x
            x = 'monty'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer:', x)

In [12]:
outer()

inner1 (before): python
inner1 (after): monty
outer: hello


Что здесь произошло, так это то, что `x` в `inner1` **замаскировал** `x` в `outer`. Но `inner2` указал Python, что `x` нелокален, поэтому первая локальная переменная в цепочке охватывающих областей видимости, которую обнаружил Python, была переменной в `inner1`, следовательно, `x` в `inner2` на самом деле ссылается на `x`, которая локальна для `inner1`

Мы можем изменить это поведение, сделав переменную `x` в `inner` также нелокальной:

In [13]:
def outer():
    x = 'hello'
    def inner1():
        nonlocal x
        x = 'python'
        def inner2():
            nonlocal x
            x = 'monty'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer:', x)

In [14]:
outer()

inner1 (before): python
inner1 (after): monty
outer: monty


In [15]:
x = 100
def outer():
    x = 'python'  # masks global x
    def inner1():
        nonlocal x  # refers to x in outer
        x = 'monty' # changed x in outer scope
        def inner2():
            global x  # refers to x in global scope
            x = 'hello'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer', x)

In [16]:
outer()
print(x)

inner1 (after): monty
outer monty
100


Но это не сработает. В `inner` Python ищет локальную переменную с именем `x`. `outer` имеет метку с именем `x`, но это глобальная переменная, а не локальная - поэтому Python не находит локальную переменную в цепочке областей видимости.

In [17]:
x = 100
def outer():
    global x
    x = 'python'

    def inner():
        nonlocal x
        x = 'monty'
    inner()

SyntaxError: no binding for nonlocal 'x' found (<ipython-input-17-3ccaec905318>, line 7)

---